## Convolutional Network in TensorFlow
In this notebook, we look at an example of CNN in TensorFlow. We will use mnist data to train our CNN on. Our CNN is structured as following
* a convolutional + relu layer
* a max pool layer
* a convolutional + relu layer
* a max pool layer
* one or two fully-connected layers


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('.', one_hot=True, reshape=False)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [11]:
print ('number of samples {}'.format(mnist.train.num_examples))
print ('number of validation {}'.format(mnist.validation.num_examples))

H = mnist.train.images[0].shape[0]
W = mnist.train.images[0].shape[1]
D = mnist.train.images[0].shape[2]

print ('input HxWxD = {}x{}x{}'.format(H,W,D))

number of samples 55000
number of validation 5000
input HxWxD = 28x28x1


In [47]:
# import library
import tensorflow as tf

# set up parameters
learning_rate = 1.0e-5
lr_adam = 1e-4
epochs = 10
batch_size = 128

# network parameters
WC1 = 5     # width of 1st conv layer's weight
HC1 = 5     # height of 1st conv layer's weight
DC1 = 32    # depth of 1st conv layer's weight

SC1 = 1     # stride for 1st conv layer
PC1 = 'SAME' # padding for 1st conv layer to keep same H & W as input

P1 = 2     # height & weight of pool window
SP1 = 2     # stride for 1st max-pool layer
OH1 = (H - P1) // SP1 + 1  
OW1 = (W - P1) // SP1 + 1

print ('after 1st conv + relu + max-pool, output has shape HxWxD: {}x{}x{}'.format(OH1, OW1, DC1) )
WC2 = 5     # width of 2nd conv layer's weight
HC2 = 5     # height of 2nd conv layer's weight
DC2 = 64    # depth of 2nd conv layer's weight
SC2 = 1     # stride for 2nd conv layer
PC2 = 'SAME' # padding to keep same H & W as input

P2 = 2
SP2 = 2
OH2 = (OH1 - P2) // SP2 + 1
OW2 = (OW1 - P2) // SP2 + 1
print ('after 1st conv + relu + max-pool, output has shape HxWxD: {}x{}x{}'.format(OH2, OW2, DC2) )

FC1_in = OH2 * OW2 * DC2
FC1_out = 1024

n_classes = 10 # mnist total classes
dropout = 0.5

# validation parameters
val_size = 256 # number of sample to compute the validation accuracy

after 1st conv + relu + max-pool, output has shape HxWxD: 14x14x32
after 1st conv + relu + max-pool, output has shape HxWxD: 7x7x64


## Define/reset weights and biases
Using above dimension input, we create weights/biases for our conv-layers & fc-layers

In [48]:
# reset to default, it will delete everything => all trained model will be lost
tf.reset_default_graph()

# now we define some weight & biases
wc1 = tf.Variable(tf.random_normal([WC1, HC1, D, DC1]), name = 'wc1')
bc1 = tf.Variable(tf.random_normal([DC1]), name = 'bc1')

wc2 = tf.Variable(tf.random_normal([WC2, HC2, DC1, DC2]), name = 'wc2')
bc2 = tf.Variable(tf.random_normal([DC2]), name = 'bc2')

wfc1 = tf.Variable(tf.random_normal([FC1_in, FC1_out]), name = 'wfc1')
bfc1 = tf.Variable(tf.random_normal([FC1_out]), name = 'bfc1')

wout = tf.Variable(tf.random_normal([FC1_out, n_classes]), name = 'wout')
bout = tf.Variable(tf.random_normal([n_classes]), name = 'bout')

for v in tf.global_variables():
    print ('defined {}'.format(v.name))

defined wc1:0
defined bc1:0
defined wc2:0
defined bc2:0
defined wfc1:0
defined bfc1:0
defined wout:0
defined bout:0


## Helpers function
We define some helper function to create conv-relu-layer/max-pool/affine-relu-dropout

In [33]:
def conv2d(x, w, b, stride_conv, padding):
    x = tf.nn.conv2d(x, w, strides = [1, stride_conv, stride_conv, 1], padding = padding)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def max_pool2d(x, kernel_size, stride_pool):
    return tf.nn.max_pool(x, 
                          ksize = [1, kernel_size, kernel_size, 1], 
                          strides = [1, stride_pool, stride_pool, 1], 
                          padding = 'SAME')

def affine_relu_dropout(x, w, b, dropout):
    x = tf.add(tf.matmul(x, w), b)
    x = tf.nn.relu(x)
    return tf.nn.dropout(x, dropout)

## Network construction
We are ready to build our first conv-net

In [41]:
def conv_net(x, dropout):
    # conv-relu-max_pool layer 1
    conv1 = conv2d(x, wc1, bc1, SC1, PC1)
    conv1 = max_pool2d(conv1, P1, SP1)
    
    # conv-relu-max_pool layer 2
    conv2 = conv2d(conv1, wc2, bc2, SC2, PC2)
    conv2 = max_pool2d(conv2, P2, SP2)
    
    # affine-relu-dropout layer 1
    fc1 = tf.reshape(conv2, [-1, FC1_in])
    fc1 = affine_relu_dropout(fc1, wfc1, bfc1, dropout)
    
    # output layer
    out = tf.add(tf.matmul(fc1, wout), bout)
    return out

In [50]:
# Now let's try it
import time
x = tf.placeholder(tf.float32, [None, H, W, D], name = 'x')
y = tf.placeholder(tf.float32, [None, n_classes], name = 'y')
keep_prob = tf.placeholder(tf.float32)

# create the conv-net
logits =conv_net(x, keep_prob)

# define the cost & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = y))

#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=lr_adam).minimize(cost)

# accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# init the variables
init = tf.global_variables_initializer()

# launch the graph
with tf.Session() as sess:
    sess.run(init)
    irun = 0
    skip_print = 100
    for epoch in range(epochs):
        ts = time.time()
        for batch in range(mnist.train.num_examples // batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {x : batch_x, y : batch_y, keep_prob : dropout})
            
            # compute loss & accuracy
            loss = sess.run(cost, feed_dict = {x : batch_x, y : batch_y, keep_prob : 1.})
            val_acc = sess.run(accuracy, feed_dict = {x : mnist.validation.images[:val_size], 
                                                      y : mnist.validation.labels[:val_size], 
                                                      keep_prob : 1.})
            
            if irun % skip_print == 0:
                print('Epoch {:>2}, Batch {:>3} -'
                      'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1, batch + 1, loss, val_acc))
            irun += 1
        print('Epoch {:>2} finished in {:>10.4f}'.format(epoch + 1, time.time() - ts))
    
    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                             y: mnist.test.labels,
                                             keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 70585.6562 Validation Accuracy: 0.132812
Epoch  1, Batch 101 -Loss: 16984.2012 Validation Accuracy: 0.367188
Epoch  1, Batch 201 -Loss:  8803.8223 Validation Accuracy: 0.593750
Epoch  1, Batch 301 -Loss:  8560.0605 Validation Accuracy: 0.730469
Epoch  1, Batch 401 -Loss:  5573.2241 Validation Accuracy: 0.769531
Epoch  1 finished in   176.9627
Epoch  2, Batch  72 -Loss:  2208.4587 Validation Accuracy: 0.785156
Epoch  2, Batch 172 -Loss:  1518.5900 Validation Accuracy: 0.812500
Epoch  2, Batch 272 -Loss:  3001.0781 Validation Accuracy: 0.812500
Epoch  2, Batch 372 -Loss:  1877.3879 Validation Accuracy: 0.828125
Epoch  2 finished in   177.4979
Epoch  3, Batch  43 -Loss:  1367.9121 Validation Accuracy: 0.847656
Epoch  3, Batch 143 -Loss:  3036.6860 Validation Accuracy: 0.843750
Epoch  3, Batch 243 -Loss:  1577.5485 Validation Accuracy: 0.843750
Epoch  3, Batch 343 -Loss:  1482.0468 Validation Accuracy: 0.859375
Epoch  3 finished in   182.3676
Epoch  4, Batch  14 